In [57]:
import tensorflow as tf

In [58]:
import math

In [59]:
def arctanh(x):
    return tf.log(tf.divide(1+x,1-x))

Initialise all of the variables

In [60]:
def inner_prod(r_in, r_out, theta_in, theta_out):
    cosine = tf.cos(theta_in - theta_out)
    radius = tf.multiply(arctanh(r_in), arctanh(r_out))
    return 4 * tf.multiply(cosine, radius)

In [61]:
def minkowski_dot(u,v):
    return tf.tensordot(u,v,1) - 2*tf.multiply(u[0],v[0])

In [62]:
def exponential(base, tangent):
    """
    Compute the exponential of `tangent` from the point `base`.
    """
    #tangent = tangent.copy()
    norm = tf.sqrt(tf.maximum(minkowski_dot(tangent, tangent), 0))
    if norm == 0:
        return base
    tangent /= norm
    return tf.cosh(norm) * base + tf.sinh(norm) * tangent

In [63]:
def tensor_inner_prod(r_example, r_sample, theta_example, theta_sample):
    r1 = arctanh(r_example)
    r2 = arctanh(r_sample)
    radius_term = r1[:, None] + r2[None, :]
    cos_term = theta_example[:, None] - theta_sample[None, :]
    return tf.squeeze(4* tf.multiply(cos_term, radius_term))

In [64]:
def nce_loss(true_logits, sampled_logits):
        true_xent = tf.nn.sigmoid_cross_entropy_with_logits(
            labels=tf.ones_like(true_logits), logits=true_logits)
        sampled_xent = tf.nn.sigmoid_cross_entropy_with_logits(
            labels=tf.zeros_like(sampled_logits), logits=sampled_logits)
        nce_loss_tensor = (tf.reduce_sum(true_xent) +
                           tf.reduce_sum(sampled_xent)) / 2
        return nce_loss_tensor

In [65]:
radius_in = tf.Variable(tf.sqrt(tf.random_uniform([5,1])))  # radius
# radius_out = tf.Variable(tf.zeros([5,1]))
radius_out = tf.Variable(tf.sqrt(tf.random_uniform([5,1])))
theta_in = tf.Variable(2*np.pi*tf.random_uniform([5,1]))  # angle
theta_out = tf.Variable(2*np.pi*tf.random_uniform([5,1]))
# theta_out = tf.Variable(tf.zeros([5,1]))
sm_b = tf.Variable(tf.zeros([5,1]))
examples = tf.Variable([1.0,2.0])
labels = tf.Variable([3.0,4.0])

In [66]:
example_radius = tf.nn.embedding_lookup(radius_in, examples)
example_theta = tf.nn.embedding_lookup(theta_in, examples)
true_radius = tf.nn.embedding_lookup(radius_out, labels)
true_theta = tf.nn.embedding_lookup(theta_out, labels)

TypeError: Value passed to parameter 'indices' has DataType float32 not in list of allowed values: int32, int64

In [67]:
sampled_ids = tf.Variable([0,1,2])
sampled_radius = tf.nn.embedding_lookup(radius_out, sampled_ids)
sampled_theta = tf.nn.embedding_lookup(theta_out, sampled_ids)
true_b = tf.nn.embedding_lookup(sm_b, labels)

TypeError: Value passed to parameter 'indices' has DataType float32 not in list of allowed values: int32, int64

In [68]:
true_logits = inner_prod(example_radius, true_radius, example_theta, true_theta) + true_b

NameError: name 'example_radius' is not defined

In [69]:
sampled_b = tf.nn.embedding_lookup(sm_b, sampled_ids)
sampled_b_vec = tf.reshape(sampled_b, sampled_ids.get_shape())

In [70]:
sampled_logits = tensor_inner_prod(example_radius, sampled_radius, example_theta, sampled_theta) + sampled_b_vec

NameError: name 'example_radius' is not defined

In [71]:
r1 = arctanh(example_radius)
r2 = arctanh(sampled_radius)
radius_term = r1[:, None] + r2[None, :]
cos_term = example_theta[:, None] - sampled_theta[None, :]
retval = 4* tf.multiply(cos_term, radius_term)

NameError: name 'example_radius' is not defined

In [72]:
loss = nce_loss(true_logits, sampled_logits)

NameError: name 'true_logits' is not defined

In [73]:
def minkowski_dist(u, v):
    """
    The distance between two points in Minkowski space
    :param u:
    :param v:
    :return:
    """
    return tf.acosh(-minkowski_dot(u, v))

In [74]:
def project_onto_tangent_space(hyperboloid_point, minkowski_tangent):
    """
    project gradients in the ambiant space onto the tangent space
    :param hyperboloid_point:
    :param minkowski_tangent:
    :return:
    """
    return minkowski_tangent + minkowski_dot(hyperboloid_point, minkowski_tangent) * hyperboloid_point

In [75]:
def exp_map(base, tangent):
    """
    Compute the exponential of the `tangent` vector from the point `base`.
    """
    # tangent = tangent.copy()
    norm = tf.sqrt(tf.maximum(minkowski_dot(tangent, tangent), 0))
    if norm == 0:
        return base
    tangent /= norm
    return tf.cosh(norm) * base + tf.sinh(norm) * tangent

In [76]:
grads = np.array([[1., 2., 0.2], [3., 4., 0.5], [4., 2., 0.6]])
x = np.eye(grads.shape[1])
x[0,0]=-1
grads.dot(x)

array([[-1. ,  2. ,  0.2],
       [-3. ,  4. ,  0.5],
       [-4. ,  2. ,  0.6]])

In [77]:
# V = tf.Variable(np.array([[1,2],[4,5],[5,2]]), dtype=tf.int32)
V = tf.Variable(np.array([1,2]), dtype=tf.int32)
x = np.eye(V.shape[0])
x[0,0]=-1
T = tf.constant(x, dtype=tf.int32)
# U = tf.matmul(V,T)
U = tf.tensordot(V,T, 1)
# y = tf.constant([math.sqrt(10001),100])
y = tf.constant([1.,0])
z = tf.constant([1.,0])
g1 = tf.constant([0.,1.])
g2 = tf.constant([0.,-1.])
g3 = tf.constant([0.,-2.])
v = tf.constant([1.,1.])
s = tf.constant([math.sqrt(5), 2])
t = tf.constant([6.2, 3.2])

In [78]:
grads = tf.Variable(np.array([[1., 2., 0.2], [3., 4., 0.5], [4., 2., 0.6]]))
true_val = np.array([[-1., 2., 0.2], [-3., 4., 0.5], [-4., 2., 0.6]])
x = np.eye(grads.shape[1])
x[0, 0] = -1.
T = tf.constant(x, dtype=tf.float64)

In [267]:
def minkowski_tensor_dot(u, v):
    """
    Minkowski dot product is the same as the Euclidean dot product, but the first element squared is subtracted
    :param u: a tensor of shape (#examples, dims)
    :param v: a tensor of shape (#examples, dims)
    :return: a scalar dot product
    """
    assert u.shape == v.shape, 'minkowski dot product not define for different shape tensors'
    try:
        temp = np.eye(u.shape[1])
    except IndexError:
        temp = np.eye(u.shape)
    temp[0, 0] = -1.
    T = tf.constant(temp, dtype=u.dtype)
    # make the first column of v negative
    v_neg = tf.matmul(v, T)
    return tf.reduce_sum(tf.multiply(u, v_neg), 1, keep_dims=True)  # keep dims for broadcasting

In [284]:
def tensor_exp_map(hyperboloid_points, tangent_grads):
    """
    Map vectors in the tangent space of the hyperboloid points back onto the hyperboloid
    :param hyperboloid_points: a tensor of points on the hyperboloid of shape (#examples, #dims)
    :param tangent_grads: a tensor of gradients on the tangent spaces of the hyperboloid_points of shape (#examples, #dims)
    :return:
    """
    # todo do we need to normalise the gradients?
    norms = tf.sqrt(tf.maximum(minkowski_tensor_dot(tangent_grads, tangent_grads), 0))
    zero = tf.constant(0, dtype=tf.float32)
    nonzero_flags = tf.squeeze(tf.not_equal(norms, zero))
    nonzero_indices = tf.squeeze(tf.where(nonzero_flags))
    nonzero_norms = tf.boolean_mask(norms, nonzero_flags)
    updated_grads = tf.boolean_mask(tangent_grads, tf.squeeze(nonzero_flags))
    updated_points = tf.boolean_mask(hyperboloid_points, nonzero_flags)
    # if norms == 0:
    #     return hyperboloid_points
    normed_grads = tf.divide(updated_grads, nonzero_norms)
    updates = tf.multiply(tf.cosh(nonzero_norms), updated_points) + tf.multiply(tf.sinh(nonzero_norms), normed_grads)
    return tf.scatter_update(hyperboloid_points, nonzero_indices, updates)

In [285]:
def project_tensors_onto_tangent_space(hyperboloid_points, ambient_gradients):
    """
    project gradients in the ambiant space onto the tangent space
    :param hyperboloid_point: A point on the hyperboloid
    :param ambient_gradient: The gradient to project
    :return: gradients in the tangent spaces of the hyperboloid points
    """
    return ambient_gradients + tf.multiply(minkowski_tensor_dot(hyperboloid_points, ambient_gradients),
                                           hyperboloid_points)

In [286]:
def transform_grads(grad):
    """
    multiply by the inverse of the Minkowski metric tensor g = diag[-1, 1,1 ... 1] to make the first element of each
    grad vector negative
    :param grad: grad matrix of shape (n_vars, embedding_dim)
    :return:
    """
    x = np.eye(grad.shape[1])
    x[0, 0] = -1.
    T = tf.constant(x, dtype=grad.dtype)
    return tf.matmul(grad, T)

In [287]:
def rsgd(grads, vecs, lr=1):
    """
    Perform the Riemannian gradient descent operation by
    1/ Transforming gradients using the Minkowski metric tensor
    2/ Projecting onto the tangent space
    3/ Applying the exponential map
    :param grads:
    :param var:
    :param lr:
    :return:
    """
    minkowski_grads = transform_grads(grads)
    tangent_grads = project_tensors_onto_tangent_space(vecs, minkowski_grads)
    return tensor_exp_map(vecs, lr * tangent_grads)

In [271]:
g1 = tf.constant([[1., 1.], [2., -1.], [3., 2.], [4., 0.]])
retval1 = np.array([[-1., 1.], [-2., -1.], [-3., 2.], [-4., 0.]])
transformed_grads = transform_grads(g1)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
tgs = sess.run(transformed_grads)
print(tgs)
assert np.array_equal(tgs, retval1)

[[-1.  1.]
 [-2. -1.]
 [-3.  2.]
 [-4.  0.]]


In [302]:
p1 = tf.Variable([[1., 0.], [1., 0.], [1., 0.], [1., 0.]])  # this the minima of the hyperboloid
g1 = tf.constant([[1., 1.], [2., -1.], [3., 2.], [4., 0.]])
retval1 = np.array([[-1.], [-1.], [-1.], [-1.]])
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
# here the tangent space is x=1
p2 = rsgd(g1, p1)
# print(sess.run(p2))
# check that the points are on the hyperboloid
# print(sess.run(p2))
norms = sess.run(minkowski_tensor_dot(p2, p2))
print(norms)
assert np.array_equal(np.around(norms, 3), retval1)

[[-0.99999976]
 [-0.99999976]
 [-0.99999714]
 [-1.        ]]


In [291]:
point = np.array([3.76219535,3.62686038])
minkowski_tensor_dot(point, point)

TypeError: 'tuple' object cannot be interpreted as an index

In [301]:
points = tf.Variable([[1., 0.], [1., 0.], [1., 0.], [1., 0.]])  # this the minima of the hyperboloid
grads = tf.Variable([[1., 1.], [2., -1.], [3., 2.], [4., 0.]])
retval1 = np.array([[-1.], [-1.], [-1.], [-1.]])
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
vals = []
for i in range(3):
#     print(sess.run(points))
    points = rsgd(grads, points)
    vals.append(points)
    # check that the points are on the hyperboloid
#     norms = sess.run(minkowski_tensor_dot(points, points))
#     print(norms)
#     try:
#         assert np.array_equal(np.around(norms, 3), retval1)
#     except AssertionError:
#         print(sess.run(points))
print(sess.run(vals))

[array([[  1.59985792e+19,   1.59985792e+19],
       [             inf,             -inf],
       [             inf,              inf],
       [  1.00000000e+00,   0.00000000e+00]], dtype=float32), array([[  1.59985792e+19,   1.59985792e+19],
       [             inf,             -inf],
       [             inf,              inf],
       [  1.00000000e+00,   0.00000000e+00]], dtype=float32), array([[  1.59985792e+19,   1.59985792e+19],
       [             inf,             -inf],
       [             inf,              inf],
       [  1.00000000e+00,   0.00000000e+00]], dtype=float32)]


In [256]:
zero = tf.constant(0, dtype=tf.float32)
p1 = tf.Variable([[1., 0.], [1., 0.], [1., 0.], [1., 0.]])  # this the minima of the hyperboloid
g1 = tf.constant([[1., 1.], [0., -1.], [0., 2.], [0., 0.]])
retval1 = np.array([[-1.], [-1.], [-1.]])
new_vals = tf.constant([4.,4.])
# norms = tf.Variable([0.,1.,0.,1.])
norms = tf.sqrt(tf.maximum(minkowski_tensor_dot(g1, g1), 0))
zero = tf.constant(0, dtype=tf.float32)
nonzero_flags = tf.squeeze(tf.not_equal(norms, zero))
nonzero_indices = tf.squeeze(tf.where(nonzero_flags))
nonzero_norms = tf.boolean_mask(norms, nonzero_flags)
updated_grads = tf.boolean_mask(g1, tf.squeeze(nonzero_flags))
updated_points = tf.boolean_mask(p1, nonzero_flags)
normed_grads = tf.divide(updated_grads, nonzero_norms)
updates = tf.multiply(tf.cosh(nonzero_norms), updated_points) + tf.multiply(tf.sinh(nonzero_norms), normed_grads)
new_vals = tf.scatter_update(p1, nonzero_indices, updates)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
# print(sess.run(tf.squeeze(where)))
# print(sess.run(tf.boolean_mask(norms, where)))
# print(sess.run(tf.boolean_mask(g1, where)))
# print(sess.run(where2))
# print(sess.run(retval))
print(sess.run(nonzero_indices))
print(sess.run(updates))
print(sess.run(new_vals))

[1 2]
[[ 1.54308069 -1.17520118]
 [ 3.76219559  3.62686038]]
[[ 1.          0.        ]
 [ 1.54308069 -1.17520118]
 [ 3.76219559  3.62686038]]


In [227]:

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(tf.where(norms==0)))
# print(sess.run(p1))
# print(sess.run(minkowski_tensor_dot(p1,p1)))
# norms = tf.sqrt(tf.maximum(minkowski_tensor_dot(p1, p1), 0))
# print(sess.run(norms))
# print(sess.run(minkowski_tensor_dot(x,y)))
# print(sess.run(tf.maximum(y, 0)))
# norms = tf.sqrt(tf.maximum(minkowski_tensor_dot(x, y), 0))

InvalidArgumentError: WhereOp : Unhandled input dimensions: 0
	 [[Node: Where = Where[_device="/job:localhost/replica:0/task:0/cpu:0"](Where/input)]]

Caused by op u'Where', defined at:
  File "/Users/ben.chamberlain/anaconda/envs/tensorflow/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/ben.chamberlain/anaconda/envs/tensorflow/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/ben.chamberlain/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/ben.chamberlain/anaconda/envs/tensorflow/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/ben.chamberlain/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/Users/ben.chamberlain/anaconda/e

In [187]:
x = np.array([[1,2],[3,4]])
x[0,1]

2

In [217]:
r1.get_shape()

TensorShape([Dimension(2), Dimension(1)])

In [224]:
tf.squeeze(example_radius).get_shape()

TensorShape([Dimension(2)])

In [219]:
retval.get_shape()

TensorShape([Dimension(2), Dimension(3), Dimension(1)])

In [229]:
sampled_logits.get_shape()

TensorShape([Dimension(2), Dimension(3)])

In [250]:
x1 = np.array([1,2])
x2 = np.array([3,4])
x3 = np.concatenate((x1.T,x2.T), axis=1)
x3

IndexError: axis 1 out of bounds [0, 1)